In [87]:
import json,folium,requests
import pandas as pd
from urllib.parse import quote

In [88]:
df = pd.read_csv('data/경기도청사및출장소현황.csv',encoding='euc-kr')
df.head()

,집계일자,시군명,구분명,전화번호안내,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,2023-03-08,안성시,안성시청,031-678-2114,17586,경기도 안성시 시청길 25,경기도 안성시 봉산동 31-3번지,37.007357,127.280313
1,2023-03-08,안성시,고삼면,031-678-3980,17505,경기도 안성시 고삼면 고삼호수로 21,경기도 안성시 고삼면 가유리 287-11번지,37.082693,127.263456
2,2023-03-08,안성시,공도읍,031-678-3610,17560,경기도 안성시 공도4로 8,경기도 안성시 공도읍 만정리 793-1번지,37.001014,127.172515
3,2023-03-08,안성시,보개면,031-678-3680,17509,경기도 안성시 보개면 보개원삼로 220,경기도 안성시 보개면 불현리 42-1번지,37.023016,127.290858
4,2023-03-08,안성시,금광면,031-678-3710,17536,경기도 안성시 금광면 금광오산로 228,경기도 안성시 금광면 내우리 20-9번지,36.997233,127.317531


In [89]:
df.isna().sum()

집계일자         0
시군명          0
구분명          0
전화번호안내      15
소재지우편번호      0
소재지도로명주소     3
소재지지번주소      0
WGS84위도      0
WGS84경도      0
dtype: int64

In [90]:
df = df[df.구분명.str.contains('도청|시청|구청')]
df.head()

,집계일자,시군명,구분명,전화번호안내,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,2023-03-08,안성시,안성시청,031-678-2114,17586,경기도 안성시 시청길 25,경기도 안성시 봉산동 31-3번지,37.007357,127.280313
17,2023-02-15,안산시,단원구청,031-481-6000,15396,경기도 안산시 단원구 중앙대로 685 (초지동),경기도 안산시 단원구 초지동 666-2번지,37.320905,126.815245
22,2023-02-15,안산시,안산시청,031-481-2000,15335,경기도 안산시 단원구 화랑로 387 (고잔동),경기도 안산시 단원구 고잔동 515번지,37.321956,126.830878
42,2023-02-15,안산시,상록구청,031-481-5000,15585,경기도 안산시 상록구 석호로 110 (사동),경기도 안산시 상록구 사동 1585번지,37.300692,126.846368
52,2023-02-14,군포시,군포시청,031-392-3000,15829,경기도 군포시 청백리길 6,경기도 군포시 금정동 844번지,37.361254,126.936150


In [91]:
df.isna().sum()

집계일자        0
시군명         0
구분명         0
전화번호안내      1
소재지우편번호     0
소재지도로명주소    0
소재지지번주소     0
WGS84위도     0
WGS84경도     0
dtype: int64

In [92]:
df = df[['구분명','소재지도로명주소','전화번호안내']]
df.head()

,구분명,소재지도로명주소,전화번호안내
0,안성시청,경기도 안성시 시청길 25,031-678-2114
17,단원구청,경기도 안산시 단원구 중앙대로 685 (초지동),031-481-6000
22,안산시청,경기도 안산시 단원구 화랑로 387 (고잔동),031-481-2000
42,상록구청,경기도 안산시 상록구 석호로 110 (사동),031-481-5000
52,군포시청,경기도 군포시 청백리길 6,031-392-3000


In [93]:
df['구분'] = df.구분명.apply(lambda x:x[-2:])
df.구분 = df.구분.apply(lambda x:'cloud' if x =='도청' else ('info-sign' if x =='시청' else 'star'))
df.head()

,구분명,소재지도로명주소,전화번호안내,구분
0,안성시청,경기도 안성시 시청길 25,031-678-2114,info-sign
17,단원구청,경기도 안산시 단원구 중앙대로 685 (초지동),031-481-6000,star
22,안산시청,경기도 안산시 단원구 화랑로 387 (고잔동),031-481-2000,info-sign
42,상록구청,경기도 안산시 상록구 석호로 110 (사동),031-481-5000,star
52,군포시청,경기도 군포시 청백리길 6,031-392-3000,info-sign


In [94]:
# df['주소'] = df.소재지도로명주소.apply(lambda x:' '.join(x.split()[:-1]))
# df.head()

In [95]:
with open('data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [96]:
base_url = "https://dapi.kakao.com/v2/local/search/address.json"
addr = ''
url = f'{base_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query='

In [97]:
header = {'Authorization':f'KakaoAK {kakao_key}'}

In [98]:
lat_list = []
lng_list = []
for i in df.index:
    url = f'{base_url}?query={quote(df.소재지도로명주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [99]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head()

,구분명,소재지도로명주소,전화번호안내,구분,위도,경도
0,안성시청,경기도 안성시 시청길 25,031-678-2114,info-sign,37.008308,127.279210
17,단원구청,경기도 안산시 단원구 중앙대로 685 (초지동),031-481-6000,star,37.319777,126.811755
22,안산시청,경기도 안산시 단원구 화랑로 387 (고잔동),031-481-2000,info-sign,37.322557,126.832714
42,상록구청,경기도 안산시 상록구 석호로 110 (사동),031-481-5000,star,37.301335,126.846410
52,군포시청,경기도 군포시 청백리길 6,031-392-3000,info-sign,37.361023,126.935316


In [100]:
df.to_csv('data/test.txt',index=False)

In [101]:
df.isna().sum()

구분명         0
소재지도로명주소    0
전화번호안내      1
구분          0
위도          0
경도          0
dtype: int64

In [102]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [103]:
map_ = folium.Map(location=[df.위도.mean(),df.경도.mean()],zoom_start=10)
for i in df.index:
    folium.Marker(
    location=[df.위도[i], df.경도[i]],                     
    popup=folium.Popup(df.전화번호안내[i],max_width=200),
    tooltip=df.구분명[i],
    icon=folium.Icon(icon=df.구분[i])
).add_to(map_)
map_